# Day 12
https://adventofcode.com/2018/day/12

In [1]:
import aocd
data = aocd.get_data(year=2018, day=12)

In [2]:
from itertools import count
import re

In [3]:
re_initial_state = re.compile(r'initial state: ([#.]+)')
re_spread_rule = re.compile(r'([#.]{5}) => ([#.])')

In [4]:
def read_initial_state(text):
    match = re_initial_state.search(text)
    if match:
        state = match.groups()[0]
        return set(n for n, char in enumerate(state) if char == '#')

In [5]:
def read_rule(pattern, result):
    return (tuple(item == '#' for item in pattern), result == '#')

In [6]:
def read_rules(text):
    return dict(read_rule(*groups) for groups in re_spread_rule.findall(text))

In [7]:
def next_generation(state, rules):
    return {n for n in range(min(state)-2, max(state)+3)
            if rules.get(tuple(n+delta in state for delta in range(-2, 3)))}

In [8]:
def state_pattern(state):
    offset = min(state)
    return tuple(s-offset for s in sorted(state))

In [9]:
def first_repeating_pattern(state, rules):
    seen = dict()
    for generation in count(1):
        pattern = state_pattern(state)
        
        first = seen.get(pattern)
        if first:
            return dict(begins=first[0], ends=generation, offset=min(state)-first[1])
        
        seen[pattern] = (generation, min(state))
        state = next_generation(state, rules)

In [10]:
def run_simulation(state, rules, generations):
    repeat = first_repeating_pattern(state, rules)
    
    # advance to the beginning of the repeating cycle (or to the total if it's low enough)
    for generation in range(min(generations, repeat['begins'])):
        state = next_generation(state, rules)
        generations -= 1

    # if the total was low enough, just return now
    if generations == 0:
        return state

    # calculate how many full cycles are needed
    cyclelength = repeat['ends'] - repeat['begins']
    cycles = generations // cyclelength
    
    # complete any remainder after the repeated cycles
    for generation in range(generations % cyclelength):
        state = next_generation(state, rules)
    
    return {x+(cycles*repeat['offset']) for x in state}

In [11]:
initial = read_initial_state(data)
rules = read_rules(data)
p1 = sum(run_simulation(initial, rules, 20))
print('Part 1: {}'.format(p1))
p2 = sum(run_simulation(initial, rules, 50_000_000_000))
print('Part 2: {}'.format(p2))

Part 1: 4110
Part 2: 2650000000466
